<a href="https://colab.research.google.com/github/sl-93/Coversation-with-my-CV-using-RAG-pipeline/blob/main/RAGPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I will be creating a RAG pipeline to communicate with my Resume.

I will be using Chromadb as a vector database and TinyLlama as an LLM in the RAG pipeline.

In [1]:
print("Installing necessary libraries...")
!pip install -q transformers langchain chromadb gradio tiktoken langchain-community sentence-transformers pypdf

print("Libraries installed successfully!")

Installing necessary libraries...
Libraries installed successfully!


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader # to read a document using langchain
from langchain.chains import RetrievalQAWithSourcesChain # to build our RAG pipeline

In [3]:
DATA_FILE_PATH = "/content/drive/MyDrive/resume/Saeed_Lotfi.pdf"

In [4]:
print(f"Attempting to load data from: {DATA_FILE_PATH}")

loader = PyPDFLoader(DATA_FILE_PATH)
raw_documents = loader.load()

print(f"Successfully loaded {len(raw_documents)} document(s)!")

Attempting to load data from: /content/drive/MyDrive/resume/Saeed_Lotfi.pdf
Successfully loaded 2 document(s)!


In [5]:
for i, doc in enumerate(raw_documents):
    print(f"Page {i+1}:")
    print(doc.page_content[:300], "\n")

Page 1:
Saeed Lotfi
AI developer at Sharif Hamrah
Pazhouhan
Master : Artiﬁcial intelligence and
robotics At K.N.Toosi University of
Technology
Personal Information
Age: 31 Years
Gender: Male
Location Province: Tehran
Location City: Tehran
Marital Status: Single
Military Service
Status:
Completed
Contact Det 

Page 2:
www.jobvision.ir
Languages
Software
Honors & Awards
Projects & Academic experiences
Papers & Books
About Me
English |  Advanced
Microsoft Word |  Advanced
 
Microsoft Excel |  Advanced
Microsoft Powerpoint |  Advanced
 
Python |  Advanced
 
GIT |  Advanced
Gitlab |  Advanced
 
Docker |  Intermediate 



## Splitting the Document into chunks with langchain text splitter

In [6]:
print("\nSplitting the document into smaller chunks...")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 150,
                                               separators=["\n\n", "\n", "."])
try:
  documents = text_splitter.split_documents(raw_documents)
  print(f"Document splitted into {len(documents)} chunks!")

except:
  raise ValueError("Error: Splitting resulted in zero.")


Splitting the document into smaller chunks...
Document splitted into 5 chunks!


In [7]:
print("\n--- Example Chunk (Chunk 1) ---")
print(documents[1].page_content)
print("\n--- Metadata for Chunk 1 ---")
print(documents[1].metadata)


--- Example Chunk (Chunk 1) ---
accuracy.
- Designing and implementing Windows applications using PyQt for data processing and
analysis.
- Working on natural language models, including ﬁne-tuning language models for
extracting features from tweets.
- Collaborating with the team to integrate machine learning models into software
environments and optimize their performance.
AI researcher July 2022 until February 2023 (7 months)
Iranian Research Organization for Science and Technology (IROST) Iran Tehran
- Research and development of a model for cardiac arrhythmia detection using
Contrastive Learning on EEG data.
- Developed a model with strong performance on imbalanced data and minimal label
requirements, ensuring efﬁciency in data processing.
AI developer October 2020 until January 2022 (1 year and 3 months)
Aicell Iran Tehran
- Fine-tuned the wave2vec model to develop a conversational AI model for Aicell social
robot, designed to assist passengers in airports.

--- Metadata for Chunk 

## Embeddings and Vector Store Creation

In [8]:
print("Initializing the Embeddings...")

# Create embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

print("Embeddings model initialized!")
print("\nCreating Chromadb vector store and embedding documents...")

# Initialize ChromaDB and add documents
vectorstore = Chroma.from_documents(documents=documents, embedding=embedding_model, persist_directory="./chromadb_store")

vector_count = vectorstore._collection.count()
print(f"ChromaDB vector store created with {vector_count} items.")


Initializing the Embeddings...


<ipython-input-8-3673041054>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

Embeddings model initialized!

Creating Chromadb vector store and embedding documents...
ChromaDB vector store created with 10 items.


In [9]:
# Retrieve the first chunk of stored data from the vector store.
stored_data = vectorstore._collection.get(include=["embeddings", "documents"], limit = 2)

print("First chunk text:\n", stored_data['documents'][1])
print("\nEmbedding Vector:\n", stored_data['embeddings'][1])
print(f"\nFull embedding has {len(stored_data['embeddings'][1])} dimensions.")

First chunk text:
 accuracy.
- Designing and implementing Windows applications using PyQt for data processing and
analysis.
- Working on natural language models, including ﬁne-tuning language models for
extracting features from tweets.
- Collaborating with the team to integrate machine learning models into software
environments and optimize their performance.
AI researcher July 2022 until February 2023 (7 months)
Iranian Research Organization for Science and Technology (IROST) Iran Tehran
- Research and development of a model for cardiac arrhythmia detection using
Contrastive Learning on EEG data.
- Developed a model with strong performance on imbalanced data and minimal label
requirements, ensuring efﬁciency in data processing.
AI developer October 2020 until January 2022 (1 year and 3 months)
Aicell Iran Tehran
- Fine-tuned the wave2vec model to develop a conversational AI model for Aicell social
robot, designed to assist passengers in airports.

Embedding Vector:
 [-3.32813002e-02 -

## Testing the Retrieval

In [10]:
print("\n--- Testing similarity search in vector store ---")
test_query = "Who is Saeed?"
print(f"Searching for documents similar to: {test_query}")

try:
  similar_docs = vectorstore.similarity_search(test_query, k=2)
  print(f"\nFound {len(similar_docs)} similar documents:")

  for i, doc in enumerate(similar_docs):
    print(f"\n--- Documents {i+1} ---")
    content_snippet = doc.page_content[:300].strip() + "..."
    source = doc.metadata.get("source", "Unknown Sources!")
    print(f"Content Snippet: {content_snippet}")
    print(f"\nSource: {source}")

except Exception as e:
  print(f"Error: {e}")


--- Testing similarity search in vector store ---
Searching for documents similar to: Who is Saeed?

Found 2 similar documents:

--- Documents 1 ---
Content Snippet: Saeed Lotfi
AI developer at Sharif Hamrah
Pazhouhan
Master : Artiﬁcial intelligence and
robotics At K.N.Toosi University of
Technology
Personal Information
Age: 31 Years
Gender: Male
Location Province: Tehran
Location City: Tehran
Marital Status: Single
Military Service
Status:
Completed
Contact Det...

Source: /content/drive/MyDrive/resume/Saeed_Lotfi.pdf

--- Documents 2 ---
Content Snippet: Saeed Lotfi
AI developer at Sharif Hamrah
Pazhouhan
Master : Artiﬁcial intelligence and
robotics At K.N.Toosi University of
Technology
Personal Information
Age: 31 Years
Gender: Male
Location Province: Tehran
Location City: Tehran
Marital Status: Single
Military Service
Status:
Completed
Contact Det...

Source: /content/drive/MyDrive/resume/Saeed_Lotfi.pdf


## Building & Testing the RAG chain using langchain

In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k":3})
print("Retriever configured successfully from vector store!")


Retriever configured successfully from vector store!


## Setup TinyLlama for generation.

In [46]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Create pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=1000,
                do_sample=True)

# Wrap in LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu


In [47]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=False  # do not return source docs
)

In [50]:
query = "Introduce Saee Lotfi"
result = qa_chain.run(query)

# Extract only the final answer
if "Helpful Answer:" in result:
    answer = result.split("Helpful Answer:")[-1].strip()
else:
    answer = result.strip()  # fallback

print("Answer:", answer)

Answer: Saeed Lotfi is an AI developer at Sharif Hamrah Pazhouhan. Saeed Lotfi is a Bachelor's degree holder in Electrical Engineering - Electronics from Semnan University and a Master's degree holder in Artificial intelligence and robotics from K.N.Toosi University of Technology. On his LinkedIn page, he has shared his experience and skills since joining the company in 2019. He has also shared his projects on Coursera, where he has completed several courses related to machine learning and deep learning.
